In [1]:
import sys 
sys.path.append("../")

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from pathlib import *
import os
import tqdm
import argparse
import json

from models.tcn import MS_TCN
from dataset.VolvoDataset import VolvoDatasetPart1, VolvoDatasetPart2
from utils.ContinuityCrossEntropyLoss import ContinuityCrossEntropyLoss
from utils.StatsComputer import StatsComputer

In [2]:
args = argparse.Namespace(
    seed=13_04_2000,
    test_only=False,
    load_model="",
    batch_size=128,
    num_epochs=30,
    learning_rate=0.0005,
    lr_scheduler_gamma=0.8,
    lr_scheduler_step=1,
    patience_epochs=7,
    disable_cuda=False,
    data_path=r"Flo_Jack\models\data",
    train_csv=r"train_gen1.csv",
    test_csv=r"public_X_test.csv",
    variants_csv=r"variants.csv",
    tcnn_weights=r"./"
)

In [3]:
args = args

np.random.seed(args.seed)
torch.manual_seed(args.seed)

### Get important paths
dataset_path = args.data_path 
variants_path = os.path.join(dataset_path, args.variants_csv)
train_data_path = os.path.join(dataset_path, args.train_csv)
test_data_path = os.path.join(dataset_path, args.test_csv)
weights_path = args.tcnn_weights
os.makedirs(weights_path, exist_ok=True)

### Get dataset and model type

train_data_path = os.path.join("data", "train_gen1.csv")
test_data_path = os.path.join("data", "public_X_test.csv")
variants_path = os.path.join("data", "variants.csv")

In [4]:
train_dataset = VolvoDatasetPart2(data_path=train_data_path, variants_path=variants_path)
processor = train_dataset.get_processor()
label_encoder = processor.risk_encoder

train_dataset, validation_dataset = train_dataset.split_train_validation()

test_dataset = VolvoDatasetPart2(data_path=test_data_path, variants_path=variants_path, test=True)
test_dataset.set_processor(processor) 

100%|██████████| 7280/7280 [00:00<00:00, 14116.36it/s]


Train: 1177 Validation: 305
Train: 962 Validation: 245


100%|██████████| 3359/3359 [00:00<00:00, 15569.19it/s]


In [5]:
train_dataset[0]

['Medium' 'Medium' 'Medium' 'Medium' 'High' 'High' 'High' 'High' 'High'
 'High']


(tensor([[0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 1.0000],
         [0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.0013, 1.0000],
         [0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.0026, 1.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.9999, 0.0092, 1.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.9999, 0.0105, 0.9999],
         [0.0000, 0.0000, 0.0000,  ..., 0.9999, 0.0119, 0.9999]]),
 tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]))

In [5]:
for elem in train_dataset:
    value, sstat, label = elem
    if label == 1:
        print(value, sstat, label)
        break

['Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low']
['Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low']
['Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low']
['Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low']
['Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low']
['High' 'High' 'High' 'High' 'High' 'High' 'High' 'High' 'High' 'High']
tensor([[0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 1.0000],
        [0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.0013, 1.0000],
        [0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.0026, 1.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.9999, 0.0092, 1.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.9999, 0.0105, 0.9999],
        [0.0000, 0.0000, 0.0000,  ..., 0.9999, 0.0119, 0.9999]]) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) 1


In [ ]:
n_features = train_dataset.get_n_features()
num_classes = train_dataset.get_n_classes()

#check if preprocess is giving some problems
assert train_dataset.get_n_features() == test_dataset.get_n_features()

model = MS_TCN(num_stages=0, 
                    num_input_channels=n_features, 
                    num_classes=num_classes)

### Get device
device = torch.device(
            "cuda" if (torch.cuda.is_available() and not args.disable_cuda) else "cpu"
        )
model.to(device)
print(f"Working on {device}")

# Load weights if necessary
if args.load_model != "":
    if not(args.load_model.endswith(".pth") or args.load_model.endswith(".pt")):
        raise Exception("Weights file should end with .pt or .pth")
    model_path = os.join.path(weights_path, args.load_model)
    print(f"Loading Model from {model_path}")
    model.load_state_dict(
        torch.load( model_path )
    )

# Create DataLoader instances for train, validation, and test sets
train_loader = DataLoader(train_dataset, 
                                batch_size=args.batch_size, 
                                collate_fn = VolvoDataset.padding_collate_fn,
                                shuffle=True,
                                num_workers=12) #pin_memory=True #consigliano
val_loader = DataLoader(validation_dataset, 
                                batch_size=args.batch_size, 
                                collate_fn = VolvoDataset.padding_collate_fn, 
                                shuffle=True,
                                num_workers=12)
test_loader =  DataLoader(test_dataset, 
                                batch_size=args.batch_size,
                                collate_fn = VolvoDataset.padding_collate_fn)

# Define criterion
print('Computing class weights...', end='')
y = label_encoder.transform( train_dataset.volvo_df[["risk_level"]].values )
y = np.argmax(y, axis=1).flatten()
y = np.array(y)[0]

weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y)
weights = weights 
criterion = ContinuityCrossEntropyLoss(
    weights=torch.Tensor(weights).to(device),
    num_classes=num_classes
    )
print('done')
print('Class weights = ', weights)
# criterion = ContinuityCrossEntropyLoss(weights=torch.Tensor([1,1,1]).to(device))